## Setup

In [2]:
import boto3

session = boto3.Session(profile_name='bach-dev', region_name='us-east-1')
boto3_bedrock = session.client(service_name='bedrock-runtime')

## Configure langchain

We begin with instantiating the LLM and the Embeddings model. Here we are using Anthropic Claude for text generation and Amazon Titan for text embedding.

Note: It is possible to choose other models available with Bedrock. You can replace the `model_id` as follows to change the model.

`llm = Bedrock(model_id="amazon.titan-text-express-v1")`


In [3]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

# - create the Anthropic Model
llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs={'max_tokens_to_sample':200})
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=boto3_bedrock)

## Data Preparation

In [4]:
import numpy as np
from langchain.text_splitter import  RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("../data/imis_guide.pdf")
pages = loader.load()

# - in our testing Character split works better with this PDF data set
text_splitter = RecursiveCharacterTextSplitter(  
    chunk_size = 1000,
    chunk_overlap  = 100,
)
docs = text_splitter.split_documents(pages)

print(f"Split {len(pages)} pages into {len(docs)} chunks.")

Split 70 pages into 136 chunks.


In [6]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)
avg_char_count_pre = avg_doc_length(pages)
avg_char_count_post = avg_doc_length(docs)
print(f'Average length among {len(pages)} pages loaded is {avg_char_count_pre} characters.')
print(f'After the split we have {len(docs)} chunks more than the original {len(pages)}.')
print(f'Average length among {len(docs)} chunks (after split) is {avg_char_count_post} characters.')

Average length among 70 pages loaded is 1451 characters.
After the split we have 136 chunks more than the original 70.
Average length among 136 chunks (after split) is 762 characters.


In [7]:
try:
    sample_embedding = np.array(bedrock_embeddings.embed_query(docs[0].page_content))
    print("Sample embedding of a document chunk: ", sample_embedding)
    print("Size of the embedding: ", sample_embedding.shape)

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

Sample embedding of a document chunk:  [ 0.10449219 -0.1484375  -0.31835938 ...  0.18554688 -0.38867188
 -0.484375  ]
Size of the embedding:  (1536,)


Following the similar pattern embeddings could be generated for the entire corpus and stored in a vector store.

This can be easily done using [PGVector](https://python.langchain.com/docs/integrations/vectorstores/pgvector) implementation inside [LangChain](https://python.langchain.com) which takes  input the embeddings model and the documents to create the entire vector store. 

Before we do that, we want to connect to our Postgres instance and enable the PGVector extension.


In [10]:
# First lets setup our connection to Postgres

import psycopg2

DRIVER="psycopg2"
HOST="serverless-database-eric-f1serverlessericbache25b-vi6utjuieajz.cluster-cma5xwwsyv4p.us-west-2.rds.amazonaws.com"
PORT="5432"
DATABASE="postgres"
USER="postgres"
PASSWORD="LIAaZgKgcgM6Br4WFXxEDzuSOaPRYLNO"

conn = psycopg2.connect(dbname=DATABASE,user=USER,host=HOST,password=PASSWORD)
cur = conn.cursor()
cur.execute('CREATE EXTENSION IF NOT EXISTS vector;')
conn.commit()

**⚠️⚠️⚠️ NOTE: it might take few minutes to run the following cell ⚠️⚠️⚠️**

The PGVector integration with langchain provides a helper function to create our database and load the emebedddings. 
The below command will take our Bedrock Embeddings, the documents, and our Postgres connection to create the vectorstore.

In [12]:
from langchain.vectorstores.pgvector import PGVector

CONNECTION_STRING = PGVector.connection_string_from_db_params(
    driver=DRIVER,
    host=HOST,
    port=PORT,
    database=DATABASE,
    user=USER,
    password=PASSWORD
)

COLLECTION_NAME = "tax_info"

db = PGVector.from_documents(
    embedding=bedrock_embeddings,
    documents=docs,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING
)

## LangChain Vector Store and Querying

Once we have the vector store created, we can use it in langchain by refrencing the store.

In [14]:
# Create a PGVector Store. Helpful if we didnt create the store in this session
vectorstore = PGVector(
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
    embedding_function=bedrock_embeddings,
)

#### We can use the similarity search method to make a query directly and return the chunks of text without any LLM generating the response.

In [15]:
query = "What is the difference between eBill and email address?"

vectorstore.similarity_search(query, k=3)  # our search query  # return 3 most relevant docs

[Document(page_content='15 | P a g e  \n  \n‘eBill’ is different from ‘Email.’ If you are making updates to eBill or an email address, ensure that you’re asking the \nMember if both should be updated.   \n                           \n \nCards & Bills  \nYou may order ad hoc C ards and Bills for members if required. You may either order Cards for the entire HH by \nselecting Household Card , or for an individual (s) by selecting Individual Card.  \n                                              \n \n \n       Household Card E xpiry  \n \nCards have a 3 year expiry and we print this replacement  date on the card.  If you order cards for any reason the system \nwill automatically check to see if the HH card expiry date is within 6 months from today.  If it is, it will not only order t he \ncard(s) you are requesting, it will advance the card expiry dat e by 3 years and order new cards for the entire HH.  \n \nIf this HH has a donor and requested  ‘Cards to:  Donor’, all 3 year replacement 

#### All of these are relevant results, telling us that the retrieval component of our systems is functioning. The next step is adding our LLM to generatively answer our question using the information provided in these retrieved contexts.

## Generative Question Answering

In generative question-answering (GQA), we pass our question to the Claude-2 but instruct it to base the answer on the information returned from our knowledge base. We can do this in LangChain easily using the RetrievalQA chain.

In [16]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())

#### Let’s try this with our earlier query:

In [17]:
qa.run(query)

c:\Users\eric.bach.CORP\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


" Based on the context provided, the main difference between eBill and email address is:\n\n- eBill refers specifically to receiving the AMA renewal bill electronically via email instead of through physical mail. This replaces the 1st paper bill that is normally sent out about 1 month before membership expiry.\n\n- Email address refers to the member's general email contact information. This could be used for contacting the member via email for various purposes, not only for eBill.\n\nSo in summary:\n\n- eBill is for receiving the renewal bill electronically \n- Email address is the member's email contact info that could be used for any email communications."

### The response we get this time is generated by our gpt-3.5-turbo LLM based on the retrieved information from our vector database.

We’re still not entirely protected from convincing yet false hallucinations by the model, they can happen, and it’s unlikely that we can eliminate the problem completely. However, we can do more to improve our trust in the answers provided.

An effective way of doing this is by adding citations to the response, allowing a user to see where the information is coming from. We can do this using a slightly different version of the RetrievalQA chain called RetrievalQAWithSourcesChain.

In [ ]:
from langchain.chains import RetrievalQAWithSourcesChain

qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever()
)

In [ ]:
qa_with_sources(query)

#### Now we have answered the question being asked but also included the source of this information being used by the LLM.

#### We’ve learned how to ground Large Language Models with source knowledge by using a vector database as our knowledge base. Using this, we can encourage accuracy in our LLM’s responses, keep source knowledge up to date, and improve trust in our system by providing citations with every answer.

We can use this embedding of the query to then fetch relevant documents.
Now our query is represented as embeddings we can do a similarity search of our query against our data store providing us with the most relevant information.

### Customisable option
In the above scenario you explored the quick and easy way to get a context-aware answer to your question. Now let's have a look at a more customizable option with the helpf of [RetrievalQA](https://python.langchain.com/en/latest/modules/chains/index_examples/vector_db_qa.html) where you can customize how the documents fetched should be added to prompt using `chain_type` parameter. Also, if you want to control how many relevant documents should be retrieved then change the `k` parameter in the cell below to see different outputs. In many scenarios you might want to know which were the source documents that the LLM used to generate the answer, you can get those documents in the output using `return_source_documents` which returns the documents that are added to the context of the LLM prompt. `RetrievalQA` also allows you to provide a custom [prompt template](https://python.langchain.com/en/latest/modules/prompts/prompt_templates/getting_started.html) which can be specific to the model.

Note: In this example we are using Anthropic Claude as the LLM under Amazon Bedrock, this particular model performs best if the inputs are provided under `Human:` and the model is requested to generate an output after `Assistant:`. In the cell below you see an example of how to control the prompt such that the LLM stays grounded and doesn't answer outside the context.

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """Human: Use the following pieces of context to provide a concise answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Assistant:"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT},
)
query = "Is it possible that I get sentenced to jail due to failure in filings?"
result = qa({"query": query})
print_ww(result["result"])

In [ ]:
result["source_documents"]

## Conclusion
Congratulations on completing this moduel on retrieval augmented generation! This is an important technique that combines the power of large language models with the precision of retrieval methods. By augmenting generation with relevant retrieved examples, the responses we recieved become more coherent, consistent and grounded. You should feel proud of learning this innovative approach. I'm sure the knowledge you've gained will be very useful for building creative and engaging language generation systems. Well done!

In the above implementation of RAG based Question Answering we have explored the following concepts and how to implement them using Amazon Bedrock and it's LangChain integration.

- Loading documents and generating embeddings to create a vector store
- Retrieving documents to the question
- Preparing a prompt which goes as input to the LLM
- Present an answer in a human friendly manner
- keep source knowledge up to date, and improve trust in our system by providing citations with every answer.

### Take-aways
- Experiment with different Vector Stores
- Leverage various models available under Amazon Bedrock to see alternate outputs
- Explore options such as persistent storage of embeddings and document chunks
- Integration with enterprise data stores

# Thank You